### 데이터 수집

In [9]:
import requests
import time
import pandas as pd
from datetime import datetime, timezone
from dotenv import load_dotenv
import os

# .env 파일 로드
load_dotenv()

# --------------------------------
# 설정
# --------------------------------
API_KEY = os.getenv('ETHEREUM_API_KEY')  # Etherscan API 키 입력
CONTRACT = os.getenv('ETHEREUM_TOKEN_CONTRACT_ADDRESS')  # PicaArtMoney 토큰 컨트랙트
START_DATE = "2020-10-13"
END_DATE   = "2024-08-07"

In [14]:
# --------------------------
# 유틸: UTC 타임스탬프 변환
# --------------------------
def to_ts_utc(dstr: str) -> int:
    y, m, d = map(int, dstr.split("-"))
    return int(datetime(y, m, d, tzinfo=timezone.utc).timestamp())

start_ts = to_ts_utc(START_DATE)
end_ts   = to_ts_utc(END_DATE)

# --------------------------
# 블록 번호 by timestamp
# --------------------------
def get_block_number_by_timestamp(ts: int, closest="before") -> int:
    url = "https://api.etherscan.io/api"
    params = {
        "module": "block",
        "action": "getblocknobytime",
        "timestamp": ts,
        "closest": closest,
        "apikey": API_KEY
    }
    r = requests.get(url, params=params, timeout=30).json()
    return int(r["result"])

start_block = get_block_number_by_timestamp(start_ts, "after")
end_block   = get_block_number_by_timestamp(end_ts, "before")
print(f"[INFO] Block range: {start_block} ~ {end_block}")

# --------------------------
# 페이지 호출 (재시도 포함)
# --------------------------
def fetch_transfers_page(page, offset, start_block, end_block):
    url = "https://api.etherscan.io/api"
    params = {
        "module": "account",
        "action": "tokentx",
        "contractaddress": CONTRACT,
        "startblock": start_block,
        "endblock": end_block,
        "page": page,
        "offset": offset,
        "sort": "asc",
        "apikey": API_KEY
    }
    # 간단 재시도
    for i in range(5):
        try:
            resp = requests.get(url, params=params, timeout=30).json()
            # status=="1"이면 결과, "0"이면 없음(또는 rate limit)
            if resp.get("status") == "1":
                return resp["result"]
            # "Max rate limit reached" 같은 메시지면 잠깐 대기 후 재시도
            if "Max rate limit reached" in resp.get("message",""):
                time.sleep(1.0 * (i+1))
                continue
            return []
        except Exception:
            time.sleep(1.0 * (i+1))
    return []

# --------------------------
# 범위 수집 (빈 청크도 로그)
# --------------------------
all_txs = []
step = 200_000        # 블록 청크 크기
offset = 10_000       # 페이지 크기
sleep_s = 0.25        # rate limit 보호
current = start_block

# (선택) 처리한 청크 목록 보관해 누락 시각화
processed_ranges = []

while current <= end_block:
    to_block = min(current + step - 1, end_block)
    processed_ranges.append((current, to_block))

    page = 1
    new_cnt = 0
    while True:
        txs = fetch_transfers_page(page, offset, current, to_block)
        if not txs:
            # 첫 페이지부터 빈 경우 → 이 청크엔 트랜잭션 없음을 명시
            if page == 1:
                print(f"[INFO] Blocks {current}–{to_block}, Page {page}, NO TX")
            break

        # 기간 필터
        for tx in txs:
            ts = int(tx.get("timeStamp", 0))
            if start_ts <= ts <= end_ts:
                all_txs.append(tx)
                new_cnt += 1

        print(f"[INFO] Blocks {current}–{to_block}, Page {page}, Added {new_cnt}, Total {len(all_txs)}")
        page += 1
        time.sleep(sleep_s)

        # 마지막 페이지(=반환건수 < offset) 판단 → 더 이상 이 청크에서 페이지 없음
        if len(txs) < offset:
            break

    current = to_block + 1  # 다음 청크로

# --------------------------
# DataFrame & 중복 제거
# --------------------------
df = pd.DataFrame(all_txs)

# 어떤 컬럼이 있는지 확인
print("[INFO] Columns:", df.columns.tolist())

# 존재하는 컬럼만으로 중복 제거
subset_cols = [c for c in ["hash", "logIndex", "transactionIndex"] if c in df.columns]
if subset_cols:
    df.drop_duplicates(subset=subset_cols, inplace=True)
else:
    df.drop_duplicates(subset=["hash"], inplace=True)

# (선택) amount 컬럼 추가
if {"value","tokenDecimal"} <= set(df.columns):
    df["amount"] = (df["value"].astype("int64") /
                    (10 ** df["tokenDecimal"].astype(int)))
else:
    df["amount"] = 0.0

out_path = "picaartmoney_transactions_full.csv"
df.to_csv(out_path, index=False)
print(f"[INFO] Finished. Total collected: {len(df)} transactions → {out_path}")

# --------------------------
# (선택) 누락 구간 시각 확인용
# --------------------------
# 빈 청크도 NO TX 로그가 찍히므로, 처리 범위가 연속적이라면 누락은 없습니다.
# 그래도 안심용으로 연속성 체크:
gaps = []
for (a1,b1),(a2,b2) in zip(processed_ranges, processed_ranges[1:]):
    if a2 != b1 + 1:
        gaps.append((b1+1, a2-1))
if gaps:
    print("[WARN] Detected block gaps in iteration (should not happen):", gaps)
else:
    print("[INFO] No block range gaps in iteration (every chunk covered).")

[INFO] Block range: 11043877 ~ 20472970
[INFO] Blocks 11043877–11243876, Page 1, Added 3828, Total 3828
[INFO] Blocks 11243877–11443876, Page 1, Added 238, Total 4066
[INFO] Blocks 11443877–11643876, Page 1, Added 321, Total 4387
[INFO] Blocks 11643877–11843876, Page 1, Added 2973, Total 7360
[INFO] Blocks 11843877–12043876, Page 1, Added 2121, Total 9481
[INFO] Blocks 12043877–12243876, Page 1, Added 5185, Total 14666
[INFO] Blocks 12243877–12443876, Page 1, Added 2273, Total 16939
[INFO] Blocks 12443877–12643876, Page 1, Added 1210, Total 18149
[INFO] Blocks 12643877–12843876, Page 1, Added 3398, Total 21547
[INFO] Blocks 12843877–13043876, Page 1, Added 1086, Total 22633
[INFO] Blocks 13043877–13243876, Page 1, Added 220, Total 22853
[INFO] Blocks 13243877–13443876, Page 1, Added 144, Total 22997
[INFO] Blocks 13443877–13643876, Page 1, Added 3338, Total 26335
[INFO] Blocks 13643877–13843876, Page 1, Added 38, Total 26373
[INFO] Blocks 13843877–14043876, Page 1, Added 10, Total 2638

### 수집 데이터 전처리

In [1]:
# ============================================================
# 0) Imports
# ============================================================
import pandas as pd
import numpy as np
import torch
import torch.nn.functional as F
from torch import nn

from torch_geometric.data import Data
from torch_geometric.nn import GATConv, GAE
from torch_geometric.utils import to_undirected, negative_sampling
from torch_geometric.transforms import RandomLinkSplit

import networkx as nx
from decimal import Decimal, getcontext
from datetime import datetime, timezone
from collections import defaultdict, Counter

# ------------------------------------------------------------
# 파일 경로
CSV_PATH = "picaartmoney_transactions_full.csv"
USE_COLS = [
    'blockNumber','timeStamp','hash','nonce','blockHash','from','contractAddress','to','value',
    'tokenName','tokenSymbol','tokenDecimal','transactionIndex','gas','gasPrice','gasUsed',
    'cumulativeGasUsed','input','confirmations'
]

# 소수 정밀도 (ETH/토큰 소수 처리 안전하게)
getcontext().prec = 50

# 상위 몇 개 이상노드 출력/저장
TOPK = 200

In [2]:
# ------------------------------------------------------------
# 0) CSV 로드 & 전처리
df = pd.read_csv(CSV_PATH, usecols=USE_COLS, dtype=str)

# 결측/공백 안전 처리
for col in ['from', 'to']:
    df[col] = df[col].fillna('').str.strip().str.lower()

# timestamp -> int / datetime
df['timeStamp'] = pd.to_numeric(df['timeStamp'], errors='coerce')
df = df.dropna(subset=['timeStamp'])
df['timestamp_dt'] = pd.to_datetime(df['timeStamp'], unit='s', utc=True)

# 숫자형 컬럼 변환
for c in ['value', 'tokenDecimal', 'gas', 'gasPrice', 'gasUsed']:
    df[c] = pd.to_numeric(df[c], errors='coerce').fillna(0)

# value → 정규화 (value / 10**tokenDecimal)
def safe_value(row):
    try:
        dec = int(row['tokenDecimal'])
        val = Decimal(int(row['value'])) / (Decimal(10) ** dec)
        return val
    except Exception:
        return Decimal(0)

df['value_float'] = df.apply(safe_value, axis=1)

# gasUsed 우선 사용 (없으면 gas)
df['gas_used'] = np.where(df['gasUsed'] > 0, df['gasUsed'], df['gas'])

# 송/수신이 비어있거나 동일지갑 자기전송은 제외(원하면 포함 가능)
df = df[(df['from'] != '') & (df['to'] != '') & (df['from'] != df['to'])]

df.head()

,blockNumber,timeStamp,hash,nonce,blockHash,from,contractAddress,to,value,tokenName,...,transactionIndex,gas,gasPrice,gasUsed,cumulativeGasUsed,input,confirmations,timestamp_dt,value_float,gas_used
0,11085582,1603098539,0x328008e17407b6ba014295bfe5069ba4f60f1296aea0...,0,0x7185066660404b22f7f1cac0862c8a7f5c1ef99e4a1f...,0x0000000000000000000000000000000000000000,0xa7e0719a65128b2f6cdbc86096753ff7d5962106,0xd28493e737fbcc957f3716143ed6e40f40357b51,1000000000,PicaArtMoney,...,251,1603895,36000000000,1603895,11088223,deprecated,12287913,2020-10-19 09:08:59+00:00,1000000000,1603895
1,11091306,1603174037,0xa8352e8094fb444e9bfa9a4a6d8011502a0f4655ad38...,1,0x6902d2e0773fd0bad116b753b98e37d61d7a4b532d71...,0xd28493e737fbcc957f3716143ed6e40f40357b51,0xa7e0719a65128b2f6cdbc86096753ff7d5962106,0xfa9b57cbe5b7bd63b436dcf205c15222b510ff27,150000000,PicaArtMoney,...,185,53110,26000000000,53110,12360057,deprecated,12282189,2020-10-20 06:07:17+00:00,150000000,53110
2,11091306,1603174037,0x36195c14399493bdf43387ded77d87d3b5e98c94a138...,2,0x6902d2e0773fd0bad116b753b98e37d61d7a4b532d71...,0xd28493e737fbcc957f3716143ed6e40f40357b51,0xa7e0719a65128b2f6cdbc86096753ff7d5962106,0x32f042b0b01f10247493a950456f4c4304d46ba5,150000000,PicaArtMoney,...,186,53110,26000000000,53110,12413167,deprecated,12282189,2020-10-20 06:07:17+00:00,150000000,53110
3,11091306,1603174037,0xbf52a2c6de897287b5c6740c891a71a5122c69566bdb...,3,0x6902d2e0773fd0bad116b753b98e37d61d7a4b532d71...,0xd28493e737fbcc957f3716143ed6e40f40357b51,0xa7e0719a65128b2f6cdbc86096753ff7d5962106,0xd4b394c60bb55f80df30dac87b6f92be34739332,300000000,PicaArtMoney,...,187,53098,26000000000,53098,12466265,deprecated,12282189,2020-10-20 06:07:17+00:00,300000000,53098
4,11091313,1603174087,0x246a4998919d4c369ff0735ea372729a9e06199055e1...,4,0xe18ff0b2e114d21a3f5cf8c2976342cea2fc558254ee...,0xd28493e737fbcc957f3716143ed6e40f40357b51,0xa7e0719a65128b2f6cdbc86096753ff7d5962106,0xc32b1345acae345c595d3bbcf62e14e5f3020456,200000000,PicaArtMoney,...,100,53098,27000000000,53098,5923677,deprecated,12282182,2020-10-20 06:08:07+00:00,200000000,53098


### 그래프데이터 생성

In [3]:
# 1) 기본엣지 그래프 (MultiDiGraph) 구축
G_base = nx.MultiDiGraph()
# 노드 추가(지갑주소)
nodes = pd.unique(pd.concat([df['from'], df['to']], ignore_index=True))
G_base.add_nodes_from(nodes)

# 엣지 추가(트랜잭션 단위)
for _, r in df.iterrows():
    G_base.add_edge(
        r['from'], r['to'],
        key=r['hash'],
        hash=r['hash'],
        timestamp=int(r['timeStamp']),
        timestamp_dt=r['timestamp_dt'],
        value_float=r['value_float'],
        gas_used=int(r['gas_used'])
    )

In [4]:
# 2) 총괄엣지 그래프 (무방향, 계좌쌍 집계)
# net_value: sum(A->B) - sum(B->A), tx_count: 총 거래 횟수, first_tx_time: 최소 시각
pair_sum_ab = defaultdict(Decimal)   # sum A->B
pair_sum_ba = defaultdict(Decimal)   # sum B->A
pair_count = Counter()
pair_first_ts = dict()

for _, r in df.iterrows():
    a, b = r['from'], r['to']
    key = tuple(sorted((a, b)))
    pair_count[key] += 1
    ts = int(r['timeStamp'])
    if key not in pair_first_ts or ts < pair_first_ts[key]:
        pair_first_ts[key] = ts
    # 방향 합계
    if a < b:
        # 저장은 작은주소,큰주소 기준으로 해두고 방향은 따로 기록
        pair_sum_ab[key] += r['value_float']  # a(작은?)→b(큰?)가 아닐 수 있어 아래에서 다시 방향판단
    else:
        pair_sum_ba[key] += r['value_float']

# 위 합산 로직은 주소 문자열 비교에 의존하므로, 실제 방향별 합계를 다시 명시적으로 구합니다.
# 더 명확하게 재집계:
pair_dir_sum = defaultdict(lambda: {'ab': Decimal(0), 'ba': Decimal(0)})
for _, r in df.iterrows():
    a, b = r['from'], r['to']
    key = tuple(sorted((a, b)))
    if (a, b) == key:
        pair_dir_sum[key]['ab'] += r['value_float']
    else:
        pair_dir_sum[key]['ba'] += r['value_float']

H_summary = nx.Graph()
H_summary.add_nodes_from(nodes)

for key in pair_count.keys():
    a, b = key
    ab = pair_dir_sum[key]['ab']
    ba = pair_dir_sum[key]['ba']
    net_value = ab - ba
    first_ts = pair_first_ts[key]
    H_summary.add_edge(
        a, b,
        net_value=float(net_value),         # CSV 저장 편의 위해 float 캐스팅(정밀 보존 원하면 str(net_value))
        tx_count=int(pair_count[key]),
        first_tx_time=int(first_ts)
    )

In [ ]:

# 3) 노드 특성 계산 (29개)
# 준비: 방향 기준별 집계
# 노드별 in/out 트랜잭션 수/금액/가스
in_count = Counter()
out_count = Counter()
in_amount_sum = defaultdict(Decimal)
out_amount_sum = defaultdict(Decimal)
in_gas_sum = Counter()
out_gas_sum = Counter()

# 노드별 타임스탬프 모음
node_timestamps = defaultdict(list)

# 노드별 in/out 이웃(상대방) 집합
in_neighbors = defaultdict(set)
out_neighbors = defaultdict(set)

for _, r in df.iterrows():
    s, t = r['from'], r['to']
    val = r['value_float']
    gasu = int(r['gas_used'])
    ts = int(r['timeStamp'])

    # 카운트/합계
    out_count[s] += 1
    in_count[t] += 1
    out_amount_sum[s] += val
    in_amount_sum[t] += val
    out_gas_sum[s] += gasu
    in_gas_sum[t] += gasu

    # 이웃
    out_neighbors[s].add(t)
    in_neighbors[t].add(s)

    # 타임스탬프
    node_timestamps[s].append(ts)
    node_timestamps[t].append(ts)

# 3-1) 유일 송/수신 상대 관련
# - "송신 트랜잭션의 유일한 상대방이 해당 노드인 노드들의 수 / 총 수량"
#   즉, X의 out_neighbors(X)가 {v}인 모든 X를 v 기준으로 집계
unique_sender_targets = defaultdict(list)  # v -> [X,...] where X sends only to v
for x, outs in out_neighbors.items():
    if len(outs) == 1:
        v = next(iter(outs))
        unique_sender_targets[v].append(x)

# 유일송신 총 수량(해당 X->v로 보낸 모든 금액 합)
unique_send_total_amount_to_v = defaultdict(Decimal)
for v, senders in unique_sender_targets.items():
    # df 필터 비용 줄이려면 사전 집계를 쓰는 것이 좋지만, 데이터 건수가 2.6만이라도 충분히 처리 가능
    mask = (df['from'].isin(senders)) & (df['to'] == v)
    if mask.any():
        unique_send_total_amount_to_v[v] = df.loc[mask, 'value_float'].sum()
    else:
        unique_send_total_amount_to_v[v] = Decimal(0)

# - "수신 트랜잭션의 유일한 상대발이 해당 노드인 노드들의 수 / 총 수량"
#   즉, Y의 in_neighbors(Y)가 {v}인 모든 Y를 v 기준으로 집계 (Y는 v에게서만 받음)
unique_receiver_sources = defaultdict(list)  # v -> [Y,...] where Y receives only from v
for y, ins in in_neighbors.items():
    if len(ins) == 1:
        v = next(iter(ins))
        unique_receiver_sources[v].append(y)

unique_recv_total_amount_from_v = defaultdict(Decimal)
for v, receivers in unique_receiver_sources.items():
    mask = (df['to'].isin(receivers)) & (df['from'] == v)
    if mask.any():
        unique_recv_total_amount_from_v[v] = df.loc[mask, 'value_float'].sum()
    else:
        unique_recv_total_amount_from_v[v] = Decimal(0)

# 3-2) 순환 거래수: 같은 SCC 안의 상대와 주고받은 트랜잭션 수
# (방향 그래프의 강결합요소 기반)
sccs = list(nx.strongly_connected_components(G_base))
comp_id = {}
for i, comp in enumerate(sccs):
    for n in comp:
        comp_id[n] = i

cycle_tx_count = Counter()
for u, v, k, d in G_base.edges(keys=True, data=True):
    if comp_id.get(u) == comp_id.get(v) and len(sccs[comp_id[u]]) > 1:
        cycle_tx_count[u] += 1
        cycle_tx_count[v] += 1

# 3-3) 양방향 상대 수: u<->v 모두 존재하는 상대 수
bidirectional_count = Counter()
# 빠른 판별: 무방향으로 변환 후, 각 이웃 중 실제로 양방향 존재하는지 체크
UG = G_base.to_undirected()
for n in G_base.nodes():
    cnt = 0
    for nbr in UG.neighbors(n):
        has_out = G_base.has_edge(n, nbr)
        has_in = G_base.has_edge(nbr, n)
        if has_out and has_in:
            cnt += 1
    bidirectional_count[n] = cnt

# 3-4) 중심성들
# Degree/Closeness/Betweenness: 무방향 그래프 기준
deg_centrality = nx.degree_centrality(UG)
# closeness는 연결요소 문제로 normalized=True 기본, 무방향에서 계산
close_centrality = nx.closeness_centrality(UG)
# betweenness: 계산 비용 큼. 노드 많으면 k-샘플링 사용 고려. 여기서는 정확 계산 시도.
bet_centrality = nx.betweenness_centrality(UG, normalized=True)

# PageRank: 방향 + 가중치(value_float)
# weight가 float이어야 해서 미리 edge attr 준비 필요 → 이미 value_float 있음
# MultiDiGraph이므로 가중치 합산 필요 → DiGraph로 합쳐서 가중치 누적
DG_weighted = nx.DiGraph()
for u, v, d in G_base.edges(data=True):
    w = float(d.get('value_float', 0))
    if w <= 0:
        continue
    if DG_weighted.has_edge(u, v):
        DG_weighted[u][v]['weight'] += w
    else:
        DG_weighted.add_edge(u, v, weight=w)

if DG_weighted.number_of_edges() > 0:
    pagerank = nx.pagerank(DG_weighted, weight='weight')
else:
    pagerank = {n: 0.0 for n in G_base.nodes()}

# 3-5) 나머지 지표들 조립
rows = []
for n in G_base.nodes():
    in_neigh = in_neighbors.get(n, set())
    out_neigh = out_neighbors.get(n, set())

    total_in_cnt = in_count.get(n, 0)
    total_out_cnt = out_count.get(n, 0)
    total_in_amt = in_amount_sum.get(n, Decimal(0))
    total_out_amt = out_amount_sum.get(n, Decimal(0))
    total_in_gas = in_gas_sum.get(n, 0)
    total_out_gas = out_gas_sum.get(n, 0)

    # 평균들(0 나눗셈 방지)
    avg_in_amt = (total_in_amt / total_in_cnt) if total_in_cnt > 0 else Decimal(0)
    avg_out_amt = (total_out_amt / total_out_cnt) if total_out_cnt > 0 else Decimal(0)
    avg_in_gas = (Decimal(total_in_gas) / total_in_cnt) if total_in_cnt > 0 else Decimal(0)
    avg_out_gas = (Decimal(total_out_gas) / total_out_cnt) if total_out_cnt > 0 else Decimal(0)

    # 가스 효율
    recv_gas_eff = (total_in_amt / Decimal(total_in_gas)) if total_in_gas > 0 else Decimal(0)
    send_gas_eff = (total_out_amt / Decimal(total_out_gas)) if total_out_gas > 0 else Decimal(0)

    # 활동 일수/일평균
    ts_list = node_timestamps.get(n, [])
    unique_days = set()
    for ts in ts_list:
        d = datetime.fromtimestamp(ts, tz=timezone.utc).date()
        unique_days.add(d)
    active_days = len(unique_days)
    total_tx_cnt = total_in_cnt + total_out_cnt
    total_amt_abs = total_in_amt + total_out_amt  # 필요시 abs 합으로 바꾸고 싶으면 수정
    tx_per_day = (total_tx_cnt / active_days) if active_days > 0 else 0
    amt_per_day = (total_amt_abs / Decimal(active_days)) if active_days > 0 else Decimal(0)

    # 유일 송/수신 관련
    uniq_send_nodes = unique_sender_targets.get(n, [])
    uniq_recv_nodes = unique_receiver_sources.get(n, [])

    uniq_send_count = len(uniq_send_nodes)
    uniq_recv_count = len(uniq_recv_nodes)
    uniq_send_amount = unique_send_total_amount_to_v.get(n, Decimal(0))
    uniq_recv_amount = unique_recv_total_amount_from_v.get(n, Decimal(0))

    # 순환 거래수/양방향 상대수
    cyc_cnt = cycle_tx_count.get(n, 0)
    bidi_cnt = bidirectional_count.get(n, 0)

    # 중심성
    deg_c = deg_centrality.get(n, 0.0)
    clo_c = close_centrality.get(n, 0.0)
    bet_c = bet_centrality.get(n, 0.0)
    pr = pagerank.get(n, 0.0)

    # 첫/마지막 트랜잭션 시각
    if ts_list:
        first_ts = min(ts_list)
        last_ts = max(ts_list)
    else:
        first_ts = None
        last_ts = None

    rows.append({
        'address': n,

        # 1~4 유일 송/수신 관련
        'unique_sender_nodes_count': uniq_send_count,                          # (1)
        'unique_receiver_nodes_count': uniq_recv_count,                        # (2)
        'unique_sender_total_amount': float(uniq_send_amount),                 # (3)
        'unique_receiver_total_amount': float(uniq_recv_amount),               # (4)

        # 5~6 순환/양방향
        'cycle_tx_count': int(cyc_cnt),                                        # (5)
        'bidirectional_counterparties': int(bidi_cnt),                         # (6)

        # 7~10 중심성
        'closeness_centrality': float(clo_c),                                  # (7)
        'betweenness_centrality': float(bet_c),                                # (8)
        'pagerank': float(pr),                                                 # (9)
        'degree_centrality': float(deg_c),                                     # (10)

        # 11~12 이웃수(방향)
        'in_neighbor_count': len(in_neigh),                                    # (11)
        'out_neighbor_count': len(out_neigh),                                  # (12)

        # 13~22 in/out 트랜잭션 집계
        'total_in_tx_count': int(total_in_cnt),                                # (13)
        'total_out_tx_count': int(total_out_cnt),                              # (14)
        'total_in_amount': float(total_in_amt),                                 # (15)
        'total_out_amount': float(total_out_amt),                               # (16)
        'avg_in_amount': float(avg_in_amt),                                     # (17)
        'avg_out_amount': float(avg_out_amt),                                   # (18)
        'total_in_gas_used': int(total_in_gas),                                 # (19)
        'total_out_gas_used': int(total_out_gas),                               # (20)
        'avg_in_gas_used': float(avg_in_gas),                                   # (21)
        'avg_out_gas_used': float(avg_out_gas),                                 # (22)

        # 23~25 일 단위
        'active_days': int(active_days),                                        # (23)
        'tx_per_day': float(tx_per_day),                                        # (24)
        'amount_per_day': float(amt_per_day),                                   # (25)

        # 26~27 가스 효율
        'recv_gas_efficiency': float(recv_gas_eff),                             # (26)
        'send_gas_efficiency': float(send_gas_eff),                             # (27)

        # 28~29 첫/마지막 거래시각 (epoch seconds)
        'first_tx_time': int(first_ts) if first_ts is not None else None,       # (28)
        'last_tx_time': int(last_ts) if last_ts is not None else None           # (29)
    })

node_features = pd.DataFrame(rows).sort_values('address').reset_index(drop=True)


In [6]:
# 4) 산출물 저장
node_features.to_csv('node_features.csv', index=False, encoding='utf-8')

import pickle
with open("G_base_multidigraph.pkl", "wb") as f:
    pickle.dump(G_base, f)
with open("H_summary_graph.pkl", "wb") as f:
    pickle.dump(H_summary, f)

print("✅ 완료: node_features.csv / G_base_multidigraph.pkl / H_summary_graph.pkl 생성")

✅ 완료: node_features.csv / G_base_multidigraph.pkl / H_summary_graph.pkl 생성


### 데이터 벡터 변환 및 학습

In [3]:
# -*- coding: utf-8 -*-
"""
Node2Vec(64d; CPU) + GAT 오토인코더(256->32->64; DEVICE) → 재구성오차 z-score → 상위 5% 이상치
- Node2Vec은 CPU에서만 수행 (edge_index도 CPU)
- DataLoader는 num_workers=0 (Windows/Jupyter PyCapsule 피클링 에러 회피)
- 임베딩만 완료 후 DEVICE로 이동하여 GAT-AE 학습
필요: torch, torch_geometric, pandas, networkx, numpy
"""

import os
import pickle
import numpy as np
import pandas as pd
import networkx as nx
import torch
from torch import nn
import torch.nn.functional as F
from torch_geometric.nn import Node2Vec, GATConv
from torch_geometric.data import Data

# ------------------------------------------------------------
# 경로/환경
NODE_FEAT_CSV = "node_features.csv"
GRAPH_PKL = "G_base_multidigraph.pkl"
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
torch.manual_seed(42)
np.random.seed(42)

In [4]:
# ------------------------------------------------------------
# 0) 노드/그래프 로드
node_df = pd.read_csv(NODE_FEAT_CSV)
addresses = node_df['address'].astype(str).tolist()
addr2idx = {a: i for i, a in enumerate(addresses)}

if not os.path.exists(GRAPH_PKL):
    raise FileNotFoundError(f"{GRAPH_PKL} 파일이 없습니다. 경로를 확인하세요.")

with open(GRAPH_PKL, "rb") as f:
    G_nx = pickle.load(f)  # networkx.MultiDiGraph

# edge_index (CPU 텐서)
edges = []
for u, v, _k in G_nx.edges(keys=True):
    if u in addr2idx and v in addr2idx:
        edges.append([addr2idx[u], addr2idx[v]])
edge_index = torch.tensor(edges, dtype=torch.long).t().contiguous()  # CPU

num_nodes = len(addresses)

# (선택) 양방향 추가: 안정성/성능 향상에 도움
# edge_index = torch.cat([edge_index, edge_index.flip(0)], dim=1)

print(f"DEVICE: {DEVICE} | nodes: {num_nodes} | edges: {edge_index.size(1)}")

# ------------------------------------------------------------
# 1) Node2Vec (CPU에서만 수행!)
#  - edge_index는 CPU에 유지
#  - num_workers=0, persistent_workers=False 로 피클링 에러 회피
data_cpu = Data(edge_index=edge_index, num_nodes=num_nodes)  # CPU 전용 Data
edge_index_cpu = data_cpu.edge_index  # alias

n2v = Node2Vec(
    edge_index_cpu,
    embedding_dim=64,
    walk_length=30,
    context_size=10,     # window
    walks_per_node=200,  # num_walk
    p=1.0, q=1.0,
    num_negative_samples=1,
    sparse=True          # SparseAdam 사용
)

n2v_loader = n2v.loader(
    batch_size=128,
    shuffle=True,
    num_workers=0,           # 중요: Windows/Jupyter 에러 회피
    persistent_workers=False # 중요
)
n2v_optimizer = torch.optim.SparseAdam(list(n2v.parameters()), lr=0.01)

def train_node2vec(epochs=5):
    n2v.train()
    for epoch in range(1, epochs + 1):
        total_loss = 0.0
        for pos_rw, neg_rw in n2v_loader:
            n2v_optimizer.zero_grad()
            # Node2Vec은 CPU에서만: .to(DEVICE) 하지 않음
            loss = n2v.loss(pos_rw, neg_rw)
            loss.backward()
            n2v_optimizer.step()
            total_loss += loss.item()
        print(f"[Node2Vec] epoch {epoch:03d} | loss {total_loss/len(n2v_loader):.4f}")

train_node2vec(epochs=5)

with torch.no_grad():
    x_init = n2v.embedding.weight.clone().detach()     # (N, 64) CPU
    x_in = x_init.to(DEVICE) # 임베딩만 DEVICE로 이동

DEVICE: cpu | nodes: 7958 | edges: 25601
[Node2Vec] epoch 001 | loss 3.0723
[Node2Vec] epoch 002 | loss 1.3729
[Node2Vec] epoch 003 | loss 0.9691
[Node2Vec] epoch 004 | loss 0.8544
[Node2Vec] epoch 005 | loss 0.8131


### GAT모델 학습 및 결과

In [5]:
# ------------------------------------------------------------
# 2) GAT 오토인코더 (DEVICE에서 수행)
class GATAutoEncoder(nn.Module):
    def __init__(self, in_dim=64, dropout=0.3):
        super().__init__()
        self.dropout = dropout
        # 64 -> (32 * 8) = 256
        self.gat1 = GATConv(in_channels=in_dim, out_channels=32, heads=8, concat=True, dropout=dropout)
        # 256 -> 32
        self.gat2 = GATConv(in_channels=256, out_channels=32, heads=1, concat=True, dropout=dropout)
        # 32 -> 64
        self.gat3 = GATConv(in_channels=32, out_channels=64, heads=1, concat=True, dropout=dropout)

    def forward(self, x, edge_index):
        x = F.dropout(x, p=self.dropout, training=self.training)
        x = self.gat1(x, edge_index)
        x = F.elu(x)

        x = F.dropout(x, p=self.dropout, training=self.training)
        x = self.gat2(x, edge_index)
        x = F.elu(x)

        x = F.dropout(x, p=self.dropout, training=self.training)
        x = self.gat3(x, edge_index)  # 최종 복원 64d
        return x

model = GATAutoEncoder(in_dim=64, dropout=0.3).to(DEVICE)
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3, weight_decay=5e-4)

# GNN용 edge_index는 DEVICE로 올림
edge_index_dev = edge_index.to(DEVICE)

# ------------------------------------------------------------
# 3) 학습 (1000 epochs, MSE 재구성오차)
def train_gat_ae(epochs=1000):
    model.train()
    for epoch in range(1, epochs + 1):
        optimizer.zero_grad()
        x_hat = model(x_in, edge_index_dev)
        loss = F.mse_loss(x_hat, x_in)
        loss.backward()
        optimizer.step()
        if epoch % 50 == 0 or epoch == 1:
            print(f"[GAT-AE] epoch {epoch:04d} | recon MSE {loss.item():.6f}")

train_gat_ae(epochs=1000)

# ------------------------------------------------------------
# 4) 재구성오차 → z-score → 상위 5% 이상치
model.eval()
with torch.no_grad():
    x_recon = model(x_in, edge_index_dev)

recon_err = torch.mean((x_recon - x_in) ** 2, dim=1).detach().cpu().numpy()
mu = recon_err.mean()
sigma = recon_err.std(ddof=1) if recon_err.size > 1 else 1e-8
z = (recon_err - mu) / (sigma if sigma > 0 else 1e-8)
z_cut = np.percentile(z, 95.0)
anom = (z > z_cut).astype(int)

print(f"상위 5% z-score 임계값: {z_cut:.4f}")
print(f"이상치 노드 수: {anom.sum()} / {len(anom)}")

# ------------------------------------------------------------
# 5) 결과 저장
out = pd.DataFrame({
    "address": addresses,
    "recon_mse": recon_err,
    "z_score": z,
    "is_anomaly_top5pct": anom
})
out.sort_values("z_score", ascending=False).to_csv("gat_ae_anomalies.csv", index=False, encoding="utf-8")
torch.save(model.state_dict(), "gat_autoencoder.pt")
np.save("node2vec_embeddings.npy", x_init.detach().cpu().numpy())  # 원본(학습 전) 임베딩 저장

print("✅ 완료: gat_ae_anomalies.csv / gat_autoencoder.pt / node2vec_embeddings.npy 생성")


[GAT-AE] epoch 0001 | recon MSE 0.251541
[GAT-AE] epoch 0050 | recon MSE 0.093097
[GAT-AE] epoch 0100 | recon MSE 0.075579
[GAT-AE] epoch 0150 | recon MSE 0.072817
[GAT-AE] epoch 0200 | recon MSE 0.073885
[GAT-AE] epoch 0250 | recon MSE 0.069703
[GAT-AE] epoch 0300 | recon MSE 0.067430
[GAT-AE] epoch 0350 | recon MSE 0.066992
[GAT-AE] epoch 0400 | recon MSE 0.066373
[GAT-AE] epoch 0450 | recon MSE 0.065949
[GAT-AE] epoch 0500 | recon MSE 0.066416
[GAT-AE] epoch 0550 | recon MSE 0.065492
[GAT-AE] epoch 0600 | recon MSE 0.065401
[GAT-AE] epoch 0650 | recon MSE 0.065288
[GAT-AE] epoch 0700 | recon MSE 0.065069
[GAT-AE] epoch 0750 | recon MSE 0.065271
[GAT-AE] epoch 0800 | recon MSE 0.065163
[GAT-AE] epoch 0850 | recon MSE 0.064657
[GAT-AE] epoch 0900 | recon MSE 0.064808
[GAT-AE] epoch 0950 | recon MSE 0.065027
[GAT-AE] epoch 1000 | recon MSE 0.064793
상위 5% z-score 임계값: 1.1053
이상치 노드 수: 398 / 7958
✅ 완료: gat_ae_anomalies.csv / gat_autoencoder.pt / node2vec_embeddings.npy 생성


### GraphSAGE 모델 학습 및 결과

In [6]:
# === GraphSAGE AutoEncoder: tail block ===
import numpy as np, pandas as pd
import torch
from torch import nn
import torch.nn.functional as F
from torch_geometric.nn import SAGEConv

# Node2Vec 임베딩을 DEVICE로, edge_index도 DEVICE로
x_in = x_init.to(DEVICE)                 # (N, 64)
edge_index_dev = edge_index.to(DEVICE)   # [2, E] long

class SAGEAutoEncoder(nn.Module):
    def __init__(self, in_dim=64, dropout=0.3, aggr="mean"):
        super().__init__()
        self.dropout = dropout
        self.s1 = SAGEConv(in_dim, 256, aggr=aggr)  # 64 -> 256
        self.s2 = SAGEConv(256, 32, aggr=aggr)      # 256 -> 32
        self.s3 = SAGEConv(32, 64, aggr=aggr)       # 32 -> 64
    def forward(self, x, edge_index):
        x = F.dropout(x, p=self.dropout, training=self.training); x = F.elu(self.s1(x, edge_index))
        x = F.dropout(x, p=self.dropout, training=self.training); x = F.elu(self.s2(x, edge_index))
        x = F.dropout(x, p=self.dropout, training=self.training); x = self.s3(x, edge_index)
        return x

model = SAGEAutoEncoder(in_dim=64, dropout=0.3, aggr="mean").to(DEVICE)
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3, weight_decay=5e-4)

# (선택) 무방향 효과를 주고 싶으면 앞에서 edge_index를 다음처럼 만들어 사용:
# edge_index = torch.cat([edge_index, edge_index.flip(0)], dim=1)

# 3) 학습 (1000 epochs, 재구성 MSE 최소화)
def train_sage_ae(epochs=1000):
    model.train()
    for epoch in range(1, epochs + 1):
        optimizer.zero_grad()
        x_hat = model(x_in, edge_index_dev)
        loss = F.mse_loss(x_hat, x_in)
        loss.backward()
        optimizer.step()
        if epoch % 50 == 0 or epoch == 1:
            print(f"[SAGE-AE] epoch {epoch:04d} | recon MSE {loss.item():.6f}")

train_sage_ae(epochs=1000)

# 4) 재구성오차 → z-score → 상위 5% 이상치
model.eval()
with torch.no_grad():
    x_recon = model(x_in, edge_index_dev)

recon_err = torch.mean((x_recon - x_in) ** 2, dim=1).detach().cpu().numpy()
mu = recon_err.mean()
sigma = recon_err.std(ddof=1) if recon_err.size > 1 else 1e-8
z = (recon_err - mu) / (sigma if sigma > 0 else 1e-8)
z_cut = np.percentile(z, 95.0)
anom = (z > z_cut).astype(int)

print(f"상위 5% z-score 임계값: {z_cut:.4f}")
print(f"이상치 노드 수: {anom.sum()} / {len(anom)}")

# 5) 결과 저장 (SAGE 파일명으로)
out = pd.DataFrame({
    "address": addresses,
    "recon_mse": recon_err,
    "z_score": z,
    "is_anomaly_top5pct": anom
})
out.sort_values("z_score", ascending=False).to_csv("sage_ae_anomalies.csv", index=False, encoding="utf-8")
torch.save(model.state_dict(), "sage_autoencoder.pt")
np.save("node2vec_embeddings.npy", x_init.detach().cpu().numpy())  # 원본 임베딩도 유지

print("✅ 완료: sage_ae_anomalies.csv / sage_autoencoder.pt / node2vec_embeddings.npy 생성")


[SAGE-AE] epoch 0001 | recon MSE 0.234432
[SAGE-AE] epoch 0050 | recon MSE 0.111355
[SAGE-AE] epoch 0100 | recon MSE 0.094500
[SAGE-AE] epoch 0150 | recon MSE 0.082080
[SAGE-AE] epoch 0200 | recon MSE 0.075716
[SAGE-AE] epoch 0250 | recon MSE 0.070295
[SAGE-AE] epoch 0300 | recon MSE 0.069124
[SAGE-AE] epoch 0350 | recon MSE 0.067108
[SAGE-AE] epoch 0400 | recon MSE 0.065676
[SAGE-AE] epoch 0450 | recon MSE 0.063934
[SAGE-AE] epoch 0500 | recon MSE 0.063027
[SAGE-AE] epoch 0550 | recon MSE 0.062900
[SAGE-AE] epoch 0600 | recon MSE 0.062647
[SAGE-AE] epoch 0650 | recon MSE 0.062210
[SAGE-AE] epoch 0700 | recon MSE 0.061324
[SAGE-AE] epoch 0750 | recon MSE 0.061219
[SAGE-AE] epoch 0800 | recon MSE 0.060815
[SAGE-AE] epoch 0850 | recon MSE 0.060919
[SAGE-AE] epoch 0900 | recon MSE 0.060038
[SAGE-AE] epoch 0950 | recon MSE 0.059871
[SAGE-AE] epoch 1000 | recon MSE 0.059955
상위 5% z-score 임계값: 1.1753
이상치 노드 수: 398 / 7958
✅ 완료: sage_ae_anomalies.csv / sage_autoencoder.pt / node2vec_embeddings

In [8]:
df_sage = pd.read_csv("sage_ae_anomalies.csv")
df_sage.head(10)

,address,recon_mse,z_score,is_anomaly_top5pct
0,0x9a9eb7e103230d3baf2bd2ddc7eae69dbb3f77b8,0.877084,15.330381,1
1,0x1938a448d105d26c40a52a1bfe99b8ca7a745ad0,0.836884,14.577360,1
2,0x167a9333bf582556f35bd4d16a7e80e191aa6476,0.818126,14.225986,1
3,0xf37b1a35647e4efc1afec5bb870e0bcbf1ac2ffc,0.790429,13.707167,1
4,0xf204a7552bb25302a70f8695c7d5edbc8e32cb85,0.788195,13.665324,1
5,0xb5f756611eddfbd63f4e8d28f2a62a401431c35a,0.750305,12.955580,1
6,0x1d5a1eaf90218e91f2bb32e42b0b02ff39827d16,0.719738,12.382997,1
7,0xaf0ae50cd011e741cdb90f624b5ff0f06fd6ef58,0.718550,12.360743,1
8,0x11784e0732270b41dd7aba1baa266f076b78f085,0.713305,12.262490,1
9,0x19095a519eccd68213b6aa7a80577337d291006e,0.689589,11.818252,1


In [9]:
df_gat = pd.read_csv("gat_ae_anomalies.csv")
df_gat.head(10)

,address,recon_mse,z_score,is_anomaly_top5pct
0,0x1938a448d105d26c40a52a1bfe99b8ca7a745ad0,1.074462,16.639975,1
1,0x9a9eb7e103230d3baf2bd2ddc7eae69dbb3f77b8,0.979735,15.079760,1
2,0x167a9333bf582556f35bd4d16a7e80e191aa6476,0.973421,14.975760,1
3,0xf37b1a35647e4efc1afec5bb870e0bcbf1ac2ffc,0.911353,13.953463,1
4,0xf204a7552bb25302a70f8695c7d5edbc8e32cb85,0.897239,13.720983,1
5,0xb5f756611eddfbd63f4e8d28f2a62a401431c35a,0.871787,13.301768,1
6,0x11784e0732270b41dd7aba1baa266f076b78f085,0.844250,12.848221,1
7,0xaf0ae50cd011e741cdb90f624b5ff0f06fd6ef58,0.839779,12.774575,1
8,0x1d5a1eaf90218e91f2bb32e42b0b02ff39827d16,0.829123,12.599068,1
9,0x19095a519eccd68213b6aa7a80577337d291006e,0.784288,11.860599,1


In [10]:
import pandas as pd

# 0) 주소 정규화(소문자/공백제거) - 선택이지만 추천
def clean_addr(s: pd.Series) -> pd.Series:
    return s.astype(str).str.strip().str.lower()

gat_addr_clean = clean_addr(df_gat['address']).rename('address')
sage_addr_clean = clean_addr(df_sage['address']).rename('address')

# ==========================================================
# 방법 1) 고유 기준(중복 제거 후 교집합 개수)
gat_unique = set(gat_addr_clean.dropna().unique())
sage_unique = set(sage_addr_clean.dropna().unique())

common_addrs = gat_unique & sage_unique
n_common_unique = len(common_addrs)

print(f"[고유 기준] 공통 address 개수: {n_common_unique}")
# 필요하면 목록 확인
# print(list(common_addrs)[:20])

# ==========================================================
# 방법 2) 행 기준(중복 포함, inner join 결과 행 수)
# - 같은 address가 여러 번 등장하면 그만큼 카운트됩니다.
merged_rows = pd.merge(
    gat_addr_clean.to_frame(),
    sage_addr_clean.to_frame(),
    on='address',
    how='inner'
)
n_common_rows = len(merged_rows)

print(f"[행 기준] 공통 address 행 수: {n_common_rows}")

# 참고: 비율까지 보고 싶다면
print(f"df_gat 고유 주소 수: {len(gat_unique)}")
print(f"df_sage 고유 주소 수: {len(sage_unique)}")
print(f"겹치는 비율(고유 기준): {n_common_unique / max(1, len(gat_unique | sage_unique)):.2%}")


[고유 기준] 공통 address 개수: 7958
[행 기준] 공통 address 행 수: 7958
df_gat 고유 주소 수: 7958
df_sage 고유 주소 수: 7958
겹치는 비율(고유 기준): 100.00%


In [11]:
import pandas as pd
from scipy.stats import spearmanr
import numpy as np

df_gat  = pd.read_csv("gat_ae_anomalies.csv")
df_sage = pd.read_csv("sage_ae_anomalies.csv")

m = df_gat[['address','z_score','recon_mse']].merge(
    df_sage[['address','z_score','recon_mse']],
    on='address', suffixes=('_gat','_sage')
)

rho, p = spearmanr(m['z_score_gat'], m['z_score_sage'])
print("Spearman rho (z_score):", rho)

N = len(m)
k = int(0.05 * N)
top_gat  = set(m.nlargest(k, 'z_score_gat')['address'])
top_sage = set(m.nlargest(k, 'z_score_sage')['address'])
inter = len(top_gat & top_sage)
jacc = inter / len(top_gat | top_sage)
print(f"Top5% overlap: {inter}/{k}, Jaccard={jacc:.3f}")

Spearman rho (z_score): 0.9961900656844646
Top5% overlap: 383/397, Jaccard=0.932
